In [1]:
import pandas as pd
import numpy as np

In [8]:
import numpy as np
import pandas as pd
from sklearn.covariance import LedoitWolf
import cvxpy as cp

# ----------------------------------------------------------
# 1. EMA-based expected return estimator
# ----------------------------------------------------------

def compute_ema_means(returns: pd.DataFrame, alpha: float = 0.1) -> pd.DataFrame:
    """
    Compute EMA of returns for each asset.
    returns: DataFrame of shape (T, 5) with columns = assets, index = dates
    alpha: smoothing parameter in (0,1]
    """
    ema = returns.copy()

    # Initialize EMA with the first row
    ema.iloc[0] = returns.iloc[0]

    for t in range(1, len(returns)):
        ema.iloc[t] = alpha * returns.iloc[t] + (1 - alpha) * ema.iloc[t - 1]

    return ema  # same shape as returns


# ----------------------------------------------------------
# 2. Single-period MVO with cash (6th asset), SHORTING ALLOWED
# ----------------------------------------------------------

def solve_mvo_with_cash(mu_5, Sigma_5, gamma: float = 5.0,
                        box_bound: float | None = None):
    """
    Mean-variance optimization with 5 risky assets + 1 cash asset.
    SHORTING ALLOWED by default.

    mu_5: np.array (5,) expected returns of risky assets
    Sigma_5: np.array (5,5) covariance of risky assets
    gamma: risk aversion
    box_bound: if not None, enforces -box_bound <= w_i <= box_bound on all 6 assets

    Returns:
        w_full: np.array (6,) weights on 5 risky + 1 cash
    """
    # 6-asset setup
    mu_full = np.zeros(6)
    mu_full[:5] = mu_5         # cash has 0 expected return

    Sigma_full = np.zeros((6, 6))
    Sigma_full[:5, :5] = Sigma_5   # cash has 0 risk and 0 correlation

    # Variable: weights on 5 risky + 1 cash
    w = cp.Variable(6)

    # Objective: max mu'w - gamma * w' Σ w
    objective = cp.Maximize(mu_full @ w - gamma * cp.quad_form(w, Sigma_full))

    # Constraints
    constraints = [cp.sum(w) == 1]  # fully invested including cash

    # Optional box constraints for leverage control
    if box_bound is not None:
        constraints += [w >= -box_bound, w <= box_bound]

    prob = cp.Problem(objective, constraints)
    prob.solve()

    if w.value is None:
        # fallback: all cash if solver fails
        return np.array([0, 0, 0, 0, 0, 1.0])

    return w.value


# ----------------------------------------------------------
# 3. End-to-end backtest loop
# ----------------------------------------------------------

def run_mvo_strategy_with_cash(
    returns: pd.DataFrame,
    ema_alpha: float = 0.1,
    cov_window: int = 60,
    gamma: float = 5.0,
    box_bound: float | None = None,
):
    """
    End-to-end MVO strategy:
      - returns: DataFrame (T, 5) of raw daily returns for 5 assets
      - ema_alpha: EMA smoothing parameter
      - cov_window: rolling window length for covariance
      - gamma: risk aversion
      - box_bound: optional per-asset bound on weights (for all 6 assets)

    Returns:
      weights_df: DataFrame of weights (6 columns: 5 assets + CASH)
      port_ret_series: Series of realized portfolio returns
    """
    ema_mu = compute_ema_means(returns, alpha=ema_alpha)

    dates = returns.index
    n_assets = returns.shape[1]
    assert n_assets == 5, "This code assumes exactly 5 risky assets."

    weight_list = []
    ret_list = []
    trade_dates = []

    # For each t, use data up to t to form weights for t+1
    for t in range(cov_window, len(returns) - 1):
        # Covariance window
        window_returns = returns.iloc[t - cov_window + 1 : t + 1]

        # Shrinkage covariance
        lw = LedoitWolf().fit(window_returns.values)
        Sigma_5 = lw.covariance_

        # EMA expected returns at time t
        mu_5 = ema_mu.iloc[t].values

        # Solve MVO (shorting allowed)
        w_full = solve_mvo_with_cash(mu_5, Sigma_5, gamma=gamma, box_bound=box_bound)

        # Realized return at t+1
        r_next_5 = returns.iloc[t + 1].values
        r_next_6 = np.append(r_next_5, 0.0)  # cash return = 0

        port_ret = np.dot(w_full, r_next_6)

        weight_list.append(w_full)
        ret_list.append(port_ret)
        trade_dates.append(dates[t + 1])

    col_names = list(returns.columns) + ["CASH"]
    weights_df = pd.DataFrame(weight_list, index=trade_dates, columns=col_names)
    port_ret_series = pd.Series(ret_list, index=trade_dates, name="portfolio_return")

    return weights_df, port_ret_series


# ----------------------------------------------------------
# 4. Example usage
# ----------------------------------------------------------


In [2]:
raw_returns = pd.read_csv("cleaned_macro.csv")

In [3]:
def clean_date_csv(df):
    df = df.copy()  # optional: avoid modifying original
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"])
        df = df.set_index("date")
        df.index = pd.to_datetime(df.index)
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df.reset_index(drop=True)
    if "date" in df.columns:
      df = df.drop(columns=["date"])
    return df

In [5]:
raw_returns = clean_date_csv(raw_returns)

In [10]:
    # Example: shorting allowed, no explicit leverage bound
weights, port_rets = run_mvo_strategy_with_cash(
        raw_returns,
        ema_alpha=0.1,
        cov_window=60,
        gamma=5.0,
        box_bound=0.5,      # e.g. set to 0.5 to enforce -0.5 <= w_i <= 0.5
    )

print("Sample weights (tail):")
print(weights.tail())

print("\nPortfolio performance summary:")
print("Mean daily return:", port_rets.mean())
print("Std daily return :", port_rets.std())
sharpe = port_rets.mean() / port_rets.std() * np.sqrt(252)
print("Annualized Sharpe :", sharpe)

Sample weights (tail):
           GLD       IEF       SPY       USO  UUP  CASH
4154  0.148776 -0.148776  0.500000  0.500000 -0.5   0.5
4155  0.231809 -0.231809  0.500000  0.500000 -0.5   0.5
4156  0.500000 -0.455609  0.500000  0.455609 -0.5   0.5
4157  0.237452 -0.237452  0.500000  0.500000 -0.5   0.5
4158  0.094473  0.500000 -0.094473  0.500000 -0.5   0.5

Portfolio performance summary:
Mean daily return: 0.00022703534982715048
Std daily return : 0.013863633758764994
Annualized Sharpe : 0.2599660745149552


In [13]:
test_raw_returns = pd.read_csv("raw_returns_test.csv")

test_raw_returns = clean_date_csv(test_raw_returns)

In [14]:
    # Example: shorting allowed, no explicit leverage bound
weights, port_rets = run_mvo_strategy_with_cash(
        test_raw_returns,
        ema_alpha=0.1,
        cov_window=60,
        gamma=5.0,
        box_bound=0.5,      # e.g. set to 0.5 to enforce -0.5 <= w_i <= 0.5
    )

print("Sample weights (tail):")
print(weights.tail())

print("\nPortfolio performance summary:")
print("Mean daily return:", port_rets.mean())
print("Std daily return :", port_rets.std())
sharpe = port_rets.mean() / port_rets.std() * np.sqrt(252)
print("Annualized Sharpe :", sharpe)

Sample weights (tail):
          GLD       IEF       SPY       USO  UUP  CASH
839  0.148776 -0.148776  0.500000  0.500000 -0.5   0.5
840  0.231809 -0.231809  0.500000  0.500000 -0.5   0.5
841  0.500000 -0.455609  0.500000  0.455609 -0.5   0.5
842  0.237452 -0.237452  0.500000  0.500000 -0.5   0.5
843  0.094473  0.500000 -0.094473  0.500000 -0.5   0.5

Portfolio performance summary:
Mean daily return: -0.00036210842949993907
Std daily return : 0.012543022255508967
Annualized Sharpe : -0.4582861287723012
